# VacationPy
----

With the weather data from WeatherPy, use jupyter-gmaps and the Google Places API for planning future vacations.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key = g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csv_file = os.path.join('..', 'WeatherPy', 'weatherpy_cities.csv')

cities_df = pd.read_csv(csv_file)
# cities_df.count()

cities_df.head()

,Unnamed: 0,City,Country,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Date
0,0,Ribeira Grande,PT,38.5167,-28.7000,68.40,73,40,12.66,2021-10-19
1,1,Punta Arenas,CL,-53.1500,-70.9167,64.51,45,75,26.46,2021-10-19
2,2,Butaritari,KI,3.0707,172.7902,82.76,62,1,6.58,2021-10-19
3,3,Port Blair,IN,11.6667,92.7500,80.67,84,18,6.69,2021-10-19
4,4,Vila Franca do Campo,PT,37.7167,-25.4333,66.61,73,0,4.50,2021-10-19


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
locations = cities_df[["Latitude", "Longitude"]]

# Convert to float
humidity = cities_df["Humidity (%)"].astype(float)

In [4]:
# Plot Heatmap
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, 
                                 dissipating = False, max_intensity = 10,
                                 point_radius = 1)

# Add layer
fig.add_layer(heat_layer)

# Display figure

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_cities_df = cities_df.loc[(cities_df["Max Temperature (F)"]>=65) & \
                                (cities_df["Max Temperature (F)"]<=85) & \
                                (cities_df["Wind Speed (mph)"]<=10) & \
                                (cities_df["Cloudiness (%)"]<=0)]

# ideal_cities_df.count()
ideal_cities_df.head()

,Unnamed: 0,City,Country,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Date
4,4,Vila Franca do Campo,PT,37.7167,-25.4333,66.61,73,0,4.50,2021-10-19
56,56,Emerald,AU,-23.5333,148.1667,73.98,54,0,6.91,2021-10-19
96,96,Santa Cruz de Tenerife,ES,28.4682,-16.2546,78.40,33,0,1.14,2021-10-19
103,103,Los Llanos de Aridane,ES,28.6585,-17.9182,71.35,33,0,8.05,2021-10-19
126,126,Williamsburg,US,37.2707,-76.7075,74.17,49,0,3.00,2021-10-19


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Store into variable named hotel_df.
# Add a "Hotel Name" column to the DataFrame.

hotel_df = ideal_cities_df[["City","Country", "Latitude", "Longitude", \
                            "Max Temperature (F)", "Humidity (%)", \
                            "Cloudiness (%)", "Wind Speed (mph)"]].reset_index(drop = True)

hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Hotel Name
0,Vila Franca do Campo,PT,37.7167,-25.4333,66.61,73,0,4.50,
1,Emerald,AU,-23.5333,148.1667,73.98,54,0,6.91,
2,Santa Cruz de Tenerife,ES,28.4682,-16.2546,78.40,33,0,1.14,
3,Los Llanos de Aridane,ES,28.6585,-17.9182,71.35,33,0,8.05,
4,Williamsburg,US,37.2707,-76.7075,74.17,49,0,3.00,


In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

# Set parameters to search for hotels with 5000 meters.
params = {
    "keyword": "hotel", 
    "radius": 5000, 
    "types": "lodging",
    "key": g_key
}
          
for index, city in hotel_df.iterrows():
    # geocoordinates from the hotel_df
    lat = city["Latitude"]
    lng = city["Longitude"]
    params["location"] = f"{lat}, {lng}"

    # in case there's no seatch result
    try:

        hotel_response = requests.get(base_url, params = params).json()
        #print(json.dumps(response_json, indent = 4, sort_keys = True))

        hotel_df.loc[index, "Hotel Name"] = hotel_response["results"][0]["name"]

    except (KeyError, IndexError):
        
        hotel_df.loc[index, "Hotel Name"] = "No results"

hotel_df.head()

,City,Country,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Hotel Name
0,Vila Franca do Campo,PT,37.7167,-25.4333,66.61,73,0,4.50,Pestana Bahia Praia
1,Emerald,AU,-23.5333,148.1667,73.98,54,0,6.91,Emerald Central Hotel
2,Santa Cruz de Tenerife,ES,28.4682,-16.2546,78.40,33,0,1.14,Hotel Escuela Santa Cruz
3,Los Llanos de Aridane,ES,28.6585,-17.9182,71.35,33,0,8.05,Hotel Hacienda de Abajo
4,Williamsburg,US,37.2707,-76.7075,74.17,49,0,3.00,DoubleTree by Hilton Hotel Williamsburg


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [9]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations)
max_temp = hotel_df["Max Temperature (F)"]

heat_layer = gmaps.heatmap_layer(locations, weights =[max(temp,0) for temp in max_temp],
                                 dissipating = False, max_intensity = 300, point_radius = 5)

marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))